In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
classes = {0:'1',1:'2',2:'3', 3:'4', 4:'5', 5:'6', 6:'7', 7:'8',
           8:'9',9:'A',10:'B',11:'C',12:'D',13:'E',14:'F',15:'G',16:'H',17:'I',
           18:'J',19:'K',20:'L',21:'M',22:'N',23:'O',24:'P',25:'Q',26:'R',27:'S',28:'T',29:'U',
           30:'V',31:'W',32:'X',33:'Y',34:'Z'}

In [74]:
background = None
accumulated_weight = 0.7
mask_color = (0.0,0.0,0.0)

ROI_top = 100
ROI_bottom = 400
ROI_right = 300
ROI_left = 600

In [75]:
max_area = 0.95
min_area = 0.0005
#contour_info = []

In [76]:
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [77]:
def segment_hand(frame, threshold=10):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    
    _ , thresholded = cv2.threshold(diff, threshold, 255,cv2.THRESH_BINARY)
    
    edges = cv2.Canny(thresholded, threshold1= 30, threshold2=250)
    #cv2.imshow('edges',edges)
    
     #Fetching contours in the frame (These contours can be of hand
    #or any other object in foreground) …

    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contours list = 0, means we didn't get any
    #contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand
        #contour_info = [(c, cv2.contourArea(c),) for c in contours[1]]
        cntrs, heirs = cv2.findContours(thresholded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        contour_info = [(c, cv2.contourArea(c),) for c in cntrs]
        #for c in contours[1]:
        #    contour_info.append((c,cv2.contourArea(c),))
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and the
  #thresholded image of hand...
    return (thresholded, hand_segment_max_cont, contour_info)

In [78]:
cam = cv2.VideoCapture(0)
num_frames = 0
fgbg = cv2.createBackgroundSubtractorMOG2()
global background

while True:
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of captured frame...
    
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)

    if num_frames < 200:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else:
        if num_frames in range(200,230):
            cv2.putText(frame_copy, "Now you can gesture", (80, 400), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        else:
            hand = segment_hand(gray_frame)

            if hand is not None:
                
                thresholded, hand_segment, contour_info = hand

                # Drawing contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right,ROI_top)], -1, (255, 0, 0),1)
                
                cv2.imshow("Thesholded Hand Image", thresholded)
                
                thresholded = cv2.resize(thresholded, (64, 64))
                thresholded = cv2.cvtColor(thresholded,cv2.COLOR_GRAY2RGB)
                thresholded = np.reshape(thresholded,(1,thresholded.shape[0],thresholded.shape[1],3))
                
                image_area = gray_frame.shape[0] * gray_frame.shape[1]  
                max_area = 0.95 * image_area
                min_area = 0.005 * image_area
                
                mask = np.zeros(roi.shape, dtype = np.uint8)

                for contour in contour_info:
                # Instead of worrying about all the smaller contours, if the area
                #  is smaller than the min, the loop will break
                    #print('y')
                    #print(contour[1])
                    if contour[1] > min_area and contour[1] < max_area:
                        # Add contour to mask
                        
                        mask = cv2.fillConvexPoly(mask, contour[0], (255,255,255))
                #cv2.imshow('mask',mask)

                mask = cv2.dilate(mask, None, iterations=10)
                mask = cv2.erode(mask, None, iterations=10)
                mask = cv2.GaussianBlur(mask, (21, 21), 0)

                cv2.imshow('mask',mask)     
                mask_stack = mask.astype('float32') / 255.0           
                roi = roi.copy().astype('float32') / 255.0


                # Blend the image and the mask
                masked = (mask_stack * roi) + ((1-mask_stack) * mask_color)
                masked = (masked * 255).astype('uint8')
                cv2.imshow("Foreground", masked)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    
    # Get the area of the image as a comparison

    
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
    
    cv2.putText(frame_copy, "Sign language recognition_ _ _",
    (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    
    cv2.imshow("Sign Detection", frame_copy)
    
    
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

In [73]:
cam.release()

In [27]:
print(max_area,min_area)

89100.0 45.0


In [ ]:
reloaded = tf.keras.models.load_model('C:/Users/suhas/Downloads/best_model_ISL.h5')

In [ ]:
thresholded = cv2.resize(img, (150, 150))

#thresholded = cv2.cvtColor(thresholded,
#                          cv2.COLOR_GRAY2RGB)

gray = cv2.cvtColor(thresholded,cv2.COLOR_BGR2GRAY)
gray_frame = cv2.GaussianBlur(gray,(5,5),cv2.BORDER_CONSTANT)
cv2.imshow('gray',gray_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

ret, mask = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)
#cv2.imshow('mask',mask)
    
edges = cv2.Canny(gray_frame, threshold1= 100, threshold2=250)

cv2.imshow('gray',edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

thresholded = np.reshape(thresholded,
                         (1,thresholded.shape[0],thresholded.shape[1],3))
print(thresholded.shape)
#pred = reloaded.predict(thresholded)

In [ ]:
print(classes[np.argmax(pred)])